The LSTM model takes in inputs from the smartphone's sensors (Accelerometer-x,-y,z and Gyroscope-x,y,z) to different driving events (Right turn, Left turn, Accelerating, Braking etc).

The model is trained and saved as a Keras file before using TFlite converter to convert into TFLite file to be used in Android Studio. 

The main issue is not being able to use the Interpreter to determine the accuracy of the TFlite model

In [1]:
# importing the neccessary libraries
# Tensorflow and Keras library
###############################################################
import tensorflow as tf
import seaborn as sns
from pylab import rcParams

from sklearn import metrics
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, LSTM, Dense, Dropout, BatchNormalization
from keras.regularizers import l2 #regularization function
from tensorflow.keras.optimizers import Adam  #perform gradient descent
from tensorflow.math import reduce_prod
print(tf.__version__)
###############################################################
# Supporting libraries
###############################################################
import pandas as pd #used to read dataset and preform dataframe operations
import numpy as np  # perform basic array operations
import sqlite3      # our raw data is in SQLite form
import matplotlib.pyplot as plt #matplotlib
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder #s scaler is for normalizing data and label encoder is to label.

print(np.__version__)
print(tf.__version__)
print(pd.__version__)
###############################################################

%matplotlib inline

sns.set(style = 'whitegrid', palette = 'muted', font_scale = 1.5)
rcParams['figure.figsize'] = 14,8


2.5.0
1.19.5
2.5.0
1.3.1


In [2]:
file = open('Safe_Agro_Data_S10_14_August.db')   ##open the file 
conn = sqlite3.connect('Safe_Agro_Data_S10_14_August.db')  ## create the cursor pointing to the table
data_df = pd.read_sql_query('SELECT * FROM SENSOR_TABLE', conn) #create pandas dataframe so we can manipulate
#data_df.insert(0, 'unit', range(1, 1+len(data_df))) # add unit column
data_df.insert(0, 'unit', range(1, 1+len(data_df)))
data_df.head() # first 5 data

,unit,TIME,ACCEL_X,ACCEL_Y,ACCEL_Z,GYRO_X,GYRO_Y,GYRO_Z,CURRENT_SPEED,ACTIVITY
0,1,06:45:33,0.213084,9.924265,-2.166753,-0.008934,-0.018097,-0.017104,0.0,Right
1,2,06:45:33,0.213084,9.924265,-2.166753,-0.008934,-0.018097,-0.017104,0.0,Right
2,3,06:45:33,0.213084,9.924265,-2.166753,-0.008934,-0.018097,-0.017104,0.0,Right
3,4,06:45:33,0.560243,8.806173,-2.741361,-0.008934,-0.018097,-0.017104,0.0,Right
4,5,06:45:33,0.193930,9.344869,-1.414973,0.036270,-0.001604,-0.024435,0.0,Right


### Creating time sequence

In [4]:
from scipy import stats

N_TIME_STEPS = 300
N_FEATURES = 6
step = 1

segments = []
labels = []

for i in range(0,len(data_df) - N_TIME_STEPS, step):
    xs = data_df['ACCEL_X'].values[i: i+N_TIME_STEPS]
    ys = data_df['ACCEL_Y'].values[i: i+N_TIME_STEPS]
    zs = data_df['ACCEL_Z'].values[i: i+N_TIME_STEPS]
    
    gxs = data_df['GYRO_X'].values[i: i+N_TIME_STEPS]
    gys = data_df['GYRO_Y'].values[i: i+N_TIME_STEPS]
    gzs = data_df['GYRO_Z'].values[i: i+N_TIME_STEPS]
    
    label = stats.mode(data_df['ACTIVITY'][i: i+N_TIME_STEPS])[0][0]
    segments.append([xs, ys, zs, gxs, gys, gzs ])
    labels.append(label)
    

In [5]:
np.array(segments).shape

(128212, 6, 300)

In [6]:
np.array(labels).shape

(128212,)

### Reshaping segments and encoding labels

In [7]:
# reshape 'segments' to proper format
reshaped_segments = np.asarray(segments, dtype = np.float32).reshape(-1, N_TIME_STEPS, N_FEATURES)
labels = np.asarray(pd.get_dummies(labels), dtype = np.float32)

In [9]:
print(reshaped_segments.shape)
print(labels.shape)

(128212, 300, 6)
(128212, 9)


### Splitting dataset into training and testing

In [10]:
from sklearn.model_selection import train_test_split
RANDOM_SEED = 42
X_train, X_test, y_train, y_test = train_test_split(reshaped_segments, labels, test_size= 0.2, random_state=RANDOM_SEED)

In [13]:
print("X_train, y_train : " , X_train.shape, y_train.shape)
print("X_test, y_test : " , X_test.shape, y_test.shape)

X_train, y_train :  (102569, 300, 6) (102569, 9)
X_test, y_test :  (25643, 300, 6) (25643, 9)


### Building LSTM layer

In [15]:
%%time
# Building the Model
model = Sequential()
model.add(LSTM(32,return_sequences = True, input_shape = (X_train.shape[1],X_train.shape[2]),
             kernel_regularizer = l2(0.001), bias_regularizer = l2(0.001), name = 'lstm_input_1'))

model.add(Dense(32, activation = 'relu', kernel_regularizer=l2(0.001), bias_regularizer = l2(0.001), name = 'Dense_2'))
model.add(tf.keras.layers.BatchNormalization())

model.add(Flatten(name = 'Flatten_3'))

model.add(Dense(32, activation = 'relu', kernel_regularizer=l2(0.001), bias_regularizer = l2(0.001), name = 'Dense_4'))
model.add(BatchNormalization())

model.add(Dense(9, activation='softmax',
               kernel_regularizer=l2(0.001), bias_regularizer = l2(0.001), name = 'output'))

# compiling the model
model.compile( loss = 'binary_crossentropy', optimizer = Adam(), metrics = ['accuracy'] )
len(model.trainable_weights)

model.summary()



Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_input_1 (LSTM)          (None, 300, 32)           4992      
_________________________________________________________________
Dense_2 (Dense)              (None, 300, 32)           1056      
_________________________________________________________________
batch_normalization_2 (Batch (None, 300, 32)           128       
_________________________________________________________________
Flatten_3 (Flatten)          (None, 9600)              0         
_________________________________________________________________
Dense_4 (Dense)              (None, 32)                307232    
_________________________________________________________________
batch_normalization_3 (Batch (None, 32)                128       
_________________________________________________________________
output (Dense)               (None, 9)                

### Model Fitting 

In [18]:
EPOCH_VALUE = 1 #test purposes

In [19]:
%%time
# Fitting the model
history = model.fit(X_train, y_train, epochs=EPOCH_VALUE, batch_size = 64, validation_split=0.3 , shuffle=False)

1122/1122 [==============================] - 93s 83ms/step - loss: 0.1284 - accuracy: 0.9835 - val_loss: 0.0761 - val_accuracy: 0.9834
Wall time: 1min 32s


### Saving model 

In [20]:
keras_file = "LSTM_model.h5"
model.save(keras_file)

### Convert Keras file into TFlite (using TFLiteConverter)

In [21]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open('LSTM_model.tflite', 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\eilha\AppData\Local\Temp\tmp9xm4_oy4\assets


INFO:tensorflow:Assets written to: C:\Users\eilha\AppData\Local\Temp\tmp9xm4_oy4\assets


### Loading TFlite model to use with Interpreter

In [23]:
interpreter = tf.lite.Interpreter('LSTM_model.tflite')
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print(input_details)
print(" ")
print(output_details)

[{'name': 'serving_default_lstm_input_1_input:0', 'index': 0, 'shape': array([  1, 300,   6]), 'shape_signature': array([ -1, 300,   6]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
 
[{'name': 'StatefulPartitionedCall:0', 'index': 52, 'shape': array([1, 9]), 'shape_signature': array([-1,  9]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]


### evaluate model with testing data (resizing the input and output nodes to match test data)

In [25]:
interpreter.resize_tensor_input(input_details[0]['index'], (25643, 300, 6))
interpreter.resize_tensor_input(output_details[0]['index'], (25643, 9))
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print(input_details)
print(" ")
print(output_details)

[{'name': 'serving_default_lstm_input_1_input:0', 'index': 0, 'shape': array([25643,   300,     6]), 'shape_signature': array([ -1, 300,   6]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
 
[{'name': 'StatefulPartitionedCall:0', 'index': 52, 'shape': array([25643,     9]), 'shape_signature': array([-1,  9]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]


### Prepare X_test

In [28]:
input_tensor = tf.convert_to_tensor(X_test, np.float32)
print(input_tensor.shape)
print(input_tensor.dtype)

(25643, 300, 6)
<dtype: 'float32'>


### set input_tensor

In [30]:
interpreter.set_tensor(input_details[0]['index'], input_tensor)
interpreter.invoke()

RuntimeError: tensorflow/lite/kernels/concatenation.cc:80 t->dims->data[d] != t0->dims->data[d] (25643 != 1)Node number 23 (CONCATENATION) failed to prepare.
Node number 5 (WHILE) failed to invoke.
